In [241]:
%load_ext kedro.extras.extensions.ipython
%reload_kedro /Users/yeungadrian/Documents/repo/TimeSeries/
# C:\Users\Adrian\Documents\repo\TimeSeries
#/Users/yeungadrian/Documents/repo/TimeSeries/

The kedro.extras.extensions.ipython extension is already loaded. To reload it, use:
  %reload_ext kedro.extras.extensions.ipython
2022-07-02 12:08:18,594 - kedro.framework.session.store - INFO - `read()` not implemented for `BaseSessionStore`. Assuming empty store.
2022-07-02 12:08:18,683 - root - INFO - ** Kedro project timeseries
2022-07-02 12:08:18,686 - root - INFO - Defined global variable `context`, `session`, `catalog` and `pipelines`


## To Do:
- Add ability to leverage
  - Cost of leverage
  - Margin
- Benchmark 

## Libraries

In [242]:
import numpy as np
import pandas as pd
from importlib import reload
import portfolio
import metrics
reload(portfolio)
reload(metrics)

from portfolio import DataLoader, Portfolio
from metrics import Metrics


## Data

In [243]:
prices = catalog.load("fund_prices")
prices

2022-07-02 12:08:18,989 - kedro.io.data_catalog - INFO - Loading data from `fund_prices` (ParquetDataSet)...


,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,2020-12-31,324.200,92.850000,91.710,3256.93,132.690000,121.410000,50.250000
1,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000
2,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
3,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
4,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
...,...,...,...,...,...,...,...,...
5322,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
5323,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
5324,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
5325,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210


In [244]:
sp500 = catalog.load("sp500")

2022-07-02 12:08:19,127 - kedro.io.data_catalog - INFO - Loading data from `sp500` (CSVDataSet)...


## Backtesting Portfolio

In [245]:
prices = DataLoader().prepare_data(prices)

In [246]:
prices

,date,ABMD,ATVI,AMD,AMZN,AAPL,DTE,EBAY
0,1999-11-01,10.250,1.050148,10.155,69.13,0.597773,13.235224,7.096464
1,1999-11-02,10.315,1.064339,10.280,66.44,0.618028,13.626682,6.801210
2,1999-11-03,10.250,1.083012,10.655,65.81,0.627654,13.626682,6.583654
3,1999-11-04,10.470,1.091975,10.315,63.06,0.643981,13.360490,6.733871
4,1999-11-05,11.315,1.148739,10.625,64.94,0.680100,13.626682,6.904808
...,...,...,...,...,...,...,...,...
5322,2020-12-24,303.410,90.960000,91.810,3172.69,131.970000,119.720000,50.120000
5323,2020-12-28,312.910,91.430000,91.600,3283.96,136.690000,119.230000,50.240000
5324,2020-12-29,320.930,91.370000,90.620,3322.00,134.870000,119.240000,50.860000
5325,2020-12-30,323.920,91.580000,92.290,3285.85,133.720000,119.660000,50.550000


In [247]:
external_data = {
    "codes": ["EBAY", "AAPL"],
    "amounts": [30,30],
    "start_date": "2013-12-27",
    "end_date": "2019-12-31",
    "timeseries": prices,
    "rebalance": True,
    "rebalance_frequency": "m"
}

In [248]:
mock_portfolio = Portfolio(**external_data)

In [249]:
projection = mock_portfolio.backtest_strategy()

In [250]:
sp500 = sp500.rename(columns={
    'Date':'date',
    'Close/Last':'market'
})

In [251]:
sp500.date = pd.to_datetime(sp500.date)

In [252]:
sp500 = sp500.sort_values(by='date').reset_index(drop=True)

In [253]:
projection = projection.merge(sp500[['date','market']],how='left', on='date')

In [254]:
""" idx = pd.date_range(external_data['start_date'],external_data['end_date'],freq='D')
projection = projection.set_index(pd.DatetimeIndex(projection.date))
projection.reindex(idx, fill_value=0) """

" idx = pd.date_range(external_data['start_date'],external_data['end_date'],freq='D')\nprojection = projection.set_index(pd.DatetimeIndex(projection.date))\nprojection.reindex(idx, fill_value=0) "

In [255]:
projection

,EBAY,AAPL,portfolio,date,market
0,30.000000,30.000000,60.000000,2013-12-27,1841.40
1,29.905869,29.701655,59.607524,2013-12-30,1841.07
2,30.379291,30.049813,60.429105,2013-12-31,1848.36
3,29.705148,29.789625,59.494773,2014-01-02,1831.98
4,29.330667,29.135269,58.465936,2014-01-03,1831.37
...,...,...,...,...,...
1508,78.763714,82.183368,160.947081,2019-12-24,3223.38
1509,78.850722,83.813910,162.664631,2019-12-26,3239.91
1510,78.459187,83.782108,162.241295,2019-12-27,3240.02
1511,77.871885,84.279366,162.151251,2019-12-30,3221.29


In [257]:
Metrics().metrics(projection)

{'arithmetic_mean_m': 0.02259260121235626,
 'arithmetic_mean_y': 0.3074700631006573,
 'geometric_mean_m': 0.019701031133584346,
 'geometric_mean_y': 0.2637882161884719,
 'std_m': 0.07728347783613997,
 'std_downside_m': 0.031759166439820234,
 'market_correlation': 0.7227839547609198,
 'alpha': 0.005690616285629266,
 'beta': 1.4363236694444261,
 'r_squared': 0.5224166452598351,
 'cagr': 0.18146117791316696,
 'sharpe_ratio': 2.218600698543962,
 'sortino_ratio': 5.398793392076743,
 'treynor_ratio': 0.11937502776062209,
 'calmar_ratio': 0.6089579965176959,
 'max_drawdown': 0.2979863618687103,
 'active_error': 0.08339548967888888,
 'tracking_error': 0.056039451167368866,
 'information_ratio': 1.4881567885063287,
 'upside_capture_ratio': 1.522279822162407,
 'downside_capture_ratio': 1.2273104016933603,
 'capture_ratio': 1.2403380758951181}